# YiJing Modul Code Review Notebook

Dieses Notebook dient zur systematischen Code-Review und zum Testen des YiJing-Moduls.

## Bereiche der Review:
1. Grundlegende Funktionalität
2. Fehlerbehandlung
3. Async Support
4. Performance
5. Ressourcenverwaltung

In [1]:
# Imports und Setup
import sys
from pathlib import Path
import logging
from pprint import pprint

# Logging Setup
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Projektpfad zum Python-Path hinzufügen
project_dir = Path.cwd()  # Da das Notebook im Root liegt
if str(project_dir) not in sys.path:
    sys.path.insert(0, str(project_dir))

## 1. Grundlegende Funktionalität

Zuerst testen wir die Kernfunktionalität des Moduls.

In [2]:
import ollama
from IPython.display import display, Markdown

# Auswahl des Modells
model_speed = {
    'dumb': 'llama3.1:latest',
    'vision': 'llama3.2-vision:latest',
    'fast': 'llama3.2:1b:latest',
    'clever': 'llama3.2:latest',
    'assistent': 'dolphin-mistral:latest'
    }

#selected_model = model_speed['dumb']
#selected_model = model_speed['vision']
#selected_model = model_speed['fast']
selected_model = model_speed['clever']
#selected_model = model_speed['uncensored']

print("Model:", selected_model)

2024-12-31 16:56:26,192 - httpx - DEBUG - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-12-31 16:56:26,194 - httpx - DEBUG - load_verify_locations cafile='/Users/johanneskaindl/miniconda3/envs/gemini_310/lib/python3.10/site-packages/certifi/cacert.pem'


Model: llama3.2:latest


In [3]:
# Import der Hauptkomponenten
from yijing import (
    YijingOracle,
    cast_hypergram,
    generiere_erweiterte_weissagung,
    formatiere_weissagung_markdown
)
from yijing.config import ModelType

SyntaxError: invalid syntax (lines.py, line 22)

In [ ]:
# Test der verschiedenen Modelle
models = {
    'genai': {
        'type': ModelType.GENAI,
        'model': 'models/gemini-1.5-flash'
    },
    'ollama': {
        'type': ModelType.OLLAMA,
        'model': selected_model
    }
}

# Wähle ein Modell für den Test
selected_model = models['ollama']  # oder 'genai'

custom_settings = {
    "model_type": selected_model['type'],
    "active_model": selected_model['model'],
    "debug": True
}

oracle = YijingOracle(custom_settings=custom_settings)

In [ ]:
# Test einer Weissagung
test_frage = "Was sagt das I Ging über meine aktuelle Situation?"
weissagung = oracle.get_response(test_frage)

# Ausgabe der Antwort
print("Strukturierte Antwort:")
pprint(weissagung)

In [10]:
from jupyter_json_viewer import display_json

In [ ]:
display_json(weissagung,
             dark_mode=True,
             )

## 2. Fehlerbehandlung

Testen der Fehlerbehandlung in verschiedenen Szenarien.

In [5]:
from pydantic import ValidationError
from yijing.enums import ModelType

In [ ]:
def test_error_cases():
    """Test verschiedener Fehlerfälle für das YijingOracle"""
    
    # Test 1: Ungültige Konfiguration
    try:
        invalid_settings = {
            "model_type": "invalid_type",
            "active_model": ""
        }
        oracle_invalid = YijingOracle(custom_settings=invalid_settings)
        assert False, "Sollte ValidationError auslösen"
    except ValidationError as e:
        print(f"✓ Erwarteter ValidationError bei ungültiger Konfiguration: {e}\n")
    
    # Test 2: Leere Frage
    oracle = YijingOracle(custom_settings={
        "model_type": ModelType.OLLAMA,
        "active_model": "llama3.2:latest"
    })
    try:
        weissagung = oracle.get_response("")
        assert False, "Sollte ValueError auslösen"
    except ValueError as e:
        print(f"✓ Erwarteter ValueError bei leerer Frage: {e}\n")

test_error_cases()

## 3. Async Support

Test der asynchronen Funktionalität.

In [ ]:
import asyncio

async def test_async_functionality():
    """Test der asynchronen Funktionen"""
    # Mehrere gleichzeitige Anfragen
    fragen = [
        "Was sagt das I Ging über meine Beziehungen?",
        "Wie entwickelt sich meine berufliche Situation?",
        "Was sollte ich in der aktuellen Situation beachten?"
    ]
    
    tasks = [oracle.get_response_async(frage) for frage in fragen]
    results = await asyncio.gather(*tasks)
    
    return results

# Ausführung des async Tests
async_results = await test_async_functionality()
print("Async Ergebnisse:")
for i, result in enumerate(async_results, 1):
    print(f"\nAntwort {i}:")
    print(result['answer'])

## 4. Performance

Analyse der Performance-Metriken.

In [ ]:
import time
import statistics

def measure_performance(n_iterations=5):
    """Misst die Performance der Hauptfunktionen"""
    timings = {
        'hypergram_gen': [],
        'oracle_response': []
    }
    
    for _ in range(n_iterations):
        # Zeitmessung für Hypergram-Generierung
        start = time.time()
        hypergram = cast_hypergram()
        timings['hypergram_gen'].append(time.time() - start)
        
        # Zeitmessung für Oracle-Antwort
        start = time.time()
        response = oracle.get_response("Test Frage")
        timings['oracle_response'].append(time.time() - start)
    
    # Statistiken berechnen
    stats = {}
    for key, values in timings.items():
        stats[key] = {
            'mean': statistics.mean(values),
            'std': statistics.stdev(values),
            'min': min(values),
            'max': max(values)
        }
    
    return stats

performance_stats = measure_performance()
print("Performance Statistiken:")
pprint(performance_stats)

## 5. Ressourcenverwaltung

Überprüfung der Ressourcenverwaltung und des Speicherverbrauchs.

In [ ]:
import psutil
import os

def monitor_resources(func):
    """Überwacht Ressourcenverbrauch einer Funktion"""
    process = psutil.Process(os.getpid())
    
    # Speicherverbrauch vor Ausführung
    mem_before = process.memory_info().rss
    
    # Funktion ausführen
    result = func()
    
    # Speicherverbrauch nach Ausführung
    mem_after = process.memory_info().rss
    mem_diff = mem_after - mem_before
    
    print(f"Speicherverbrauch: {mem_diff / 1024 / 1024:.2f} MB")
    return result

# Test der Ressourcennutzung
def test_function():
    return oracle.get_response("Test Frage")

result = monitor_resources(test_function)

## Zusammenfassung

Basierend auf den Tests können wir folgende Aspekte analysieren:
1. Funktionalität und Korrektheit
2. Fehlerbehandlung und Robustheit
3. Performance und Skalierbarkeit
4. Ressourceneffizienz
5. Code-Qualität und Wartbarkeit